In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x = mnist.train.images

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [3]:
# Parameters
num_steps = 50 # Total steps to train
batch_size = 1024 # The number of samples per batch
k = 25 # The number of clusters
num_classes = 10 # The 10 digits
num_features = 784 # Each image is 28x28 pixels

# Input images
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Labels (for assigning a label to a centroid and testing)
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

# K-Means Parameters
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)

In [4]:
# Build KMeans graph
(all_scores, cluster_idx, scores, cluster_centers_initialized,init_op,train_op) = kmeans.training_graph()
cluster_idx = cluster_idx[0] # fix for cluster_idx being a tuple
avg_distance = tf.reduce_mean(scores)

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()

Instructions for updating:
dim is deprecated, use axis instead


In [5]:
# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars, feed_dict={X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})

# Training
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                         feed_dict={X: full_data_x})

    print("Step %i, Avg Distance: %f" % (i, d))

Step 1, Avg Distance: 0.341471
Step 2, Avg Distance: 0.234025
Step 3, Avg Distance: 0.227649
Step 4, Avg Distance: 0.225228
Step 5, Avg Distance: 0.223952
Step 6, Avg Distance: 0.223155
Step 7, Avg Distance: 0.222603
Step 8, Avg Distance: 0.222190
Step 9, Avg Distance: 0.221868
Step 10, Avg Distance: 0.221609
Step 11, Avg Distance: 0.221393
Step 12, Avg Distance: 0.221210
Step 13, Avg Distance: 0.221052
Step 14, Avg Distance: 0.220915
Step 15, Avg Distance: 0.220792
Step 16, Avg Distance: 0.220680
Step 17, Avg Distance: 0.220580
Step 18, Avg Distance: 0.220488
Step 19, Avg Distance: 0.220405
Step 20, Avg Distance: 0.220328
Step 21, Avg Distance: 0.220257
Step 22, Avg Distance: 0.220190
Step 23, Avg Distance: 0.220128
Step 24, Avg Distance: 0.220070
Step 25, Avg Distance: 0.220015
Step 26, Avg Distance: 0.219963
Step 27, Avg Distance: 0.219913
Step 28, Avg Distance: 0.219865
Step 29, Avg Distance: 0.219820
Step 30, Avg Distance: 0.219776
Step 31, Avg Distance: 0.219735
Step 32, Avg Dist

In [6]:
# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
# Assign the most frequent label to the centroid
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

# Evaluation ops
# Lookup: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Compute accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Test Model
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.7127
